# When Your Data Breaks the Rules: A Gentle Introduction to Generalized Least Squares

**A tutorial for `python-gls` — the first Python library for GLS with learned correlation structures**

---

## Who Is This For?

If you've taken an introductory statistics course and learned about linear regression (OLS), you're ready for this tutorial. We'll use a concrete, everyday example to show you *why* the standard regression you learned has a dangerous blind spot — and how Generalized Least Squares (GLS) fixes it.

By the end, you'll understand:
- Why Ordinary Least Squares (OLS) can give you **wrong answers** when your data has a natural grouping
- How **correlation structures** capture real patterns in your data
- Why **unstructured correlations** give you the most flexibility to let your data speak
- How to use `python-gls` to fit these models in Python

## 1. The Freshman GPA Problem

Imagine you're the dean of a university, and you want to answer a simple question:

> **Does participating in a study group improve students' GPAs?**

You have data on 60 students tracked across 4 semesters (freshman year through sophomore year). For each student, at each semester, you recorded:
- Their **GPA** that semester (the outcome you care about)
- Whether they participated in a **study group** (yes/no)
- Their **course load** — how many credit hours they took

Your instinct from intro stats: run a linear regression. GPA is the dependent variable, study group participation and course load are the predictors. Easy, right?

**Not so fast.** There's a hidden problem in this data that, if ignored, will give you the wrong answer — potentially leading the university to waste resources on an ineffective program, or worse, to cancel a program that actually works.

## 2. A Quick Refresher: What OLS Assumes

In your intro stats course, you learned that Ordinary Least Squares (OLS) regression finds the line (or plane) that minimizes the sum of squared errors:

$$y_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \varepsilon_i$$

where $\varepsilon_i$ is the "error" or "residual" — the part of $y$ that your predictors can't explain.

For the math to work and give you valid p-values and confidence intervals, OLS requires three key assumptions about these errors:

| Assumption | Plain English | Formal |
|---|---|---|
| **Independence** | Knowing one student's error tells you nothing about another's | $\text{Cov}(\varepsilon_i, \varepsilon_j) = 0$ for $i \ne j$ |
| **Constant variance** | The spread of errors is the same everywhere | $\text{Var}(\varepsilon_i) = \sigma^2$ for all $i$ |
| **Normality** | Errors follow a bell curve | $\varepsilon_i \sim N(0, \sigma^2)$ |

The **independence** assumption is the one that gets violated most often in real-world data — and it's the most dangerous to ignore.

## 3. Why Our GPA Data Breaks the Rules

Look at our data again. We have 60 students $\times$ 4 semesters = 240 observations. OLS would treat all 240 observations as independent — as if each row in our spreadsheet came from a different, unrelated student.

But that's obviously wrong. Consider:

- **Student A** had a 3.8 GPA last semester. What's your best guess for their GPA this semester? Probably close to 3.8 — not the class average of 3.2.
- **Student B** struggled with a 2.1 last semester. Their next semester's GPA will likely be closer to 2.1 than to 3.2.

The four GPA measurements from the same student are **correlated**. They share something — that student's underlying ability, motivation, study habits, and life circumstances — that makes them more similar to each other than to a random student's GPA.

As Snijders and Bosker (2012, Ch. 2) put it, this dependency between observations within groups is not just a statistical nuisance — it's an *interesting phenomenon* that reflects real structure in the world. Students within the same student are like pupils within the same school: they share a common environment that makes their outcomes correlated.

### The Intraclass Correlation

How correlated are observations within the same student? The **intraclass correlation coefficient** (ICC) quantifies this (Snijders & Bosker, 2012, Sec. 3.3):

$$\rho_I = \frac{\tau^2}{\tau^2 + \sigma^2}$$

where $\tau^2$ is the variance *between* students and $\sigma^2$ is the variance *within* students across semesters. If $\rho_I = 0.5$, it means that half of the total variation in GPA is due to stable differences between students.

In educational data, the ICC is typically between 0.10 and 0.25 for students nested in schools (Hedges & Hedberg, 2007). For **repeated measures on the same person**, it's often much higher — frequently 0.4 to 0.7 — because a person is much more similar to themselves over time than to a random other person.

## 4. What Goes Wrong When You Ignore Correlation

"OK," you might think, "so the data points are correlated. Does it really matter for my regression?"

**Yes. A lot.** Here's what happens:

### Your Standard Errors Are Wrong

OLS computes standard errors under the assumption that all 240 observations provide independent information. But they don't — the 4 observations from the same student are partly redundant. The **effective sample size** is smaller than 240.

The design effect formula (Snijders & Bosker, 2012, Sec. 3.4) tells us how much information we lose:

$$\text{design effect} = 1 + (n - 1) \cdot \rho_I$$

where $n$ is the group size (4 semesters) and $\rho_I$ is the ICC. If $\rho_I = 0.5$:

$$\text{design effect} = 1 + (4 - 1) \times 0.5 = 2.5$$

This means our 240 observations carry only as much information as $240 / 2.5 = 96$ truly independent observations. OLS *thinks* it has 240 independent data points, so it computes standard errors that are **too small** — making effects look more "statistically significant" than they really are.

### You Find Effects That Don't Exist

Because the standard errors are too small, your p-values are too small, and your confidence intervals are too narrow. You'll reject the null hypothesis too often — a problem called **inflated Type I error** (Dorman, 2008). You might conclude that the study group program "works" when it doesn't, or that a drug is effective when it isn't.

### You Miss the Structure in Your Data

Perhaps even more importantly, the correlation between repeated measurements is **informative**. It tells you something about the underlying process. Does a student's GPA this semester depend on last semester's GPA? How strong is that carryover? These are interesting questions that OLS simply ignores.

## 5. A Simulation

Time to see the problem with actual code. We'll generate data where the within-student correlation has a **non-monotonic** pattern that no simple structure can capture.

Here's the idea. A student's GPA in one semester is correlated with their GPA in other semesters, but not uniformly:

- **Semesters 1–2** are highly correlated ($r = 0.70$) because students are still adjusting to college
- **Semesters 3–4** are highly correlated ($r = 0.65$) once students have settled into their major
- **Semesters 2–3** are weakly correlated ($r = 0.30$), perhaps because students changed majors or went abroad

This pattern is non-monotonic: correlation doesn't simply decay with time distance. AR(1) and compound symmetry can't capture it. Only the unstructured model can.

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

# Study design
n_students = 60
n_semesters = 4
N = n_students * n_semesters

# Student-level variables
student_id = np.repeat(np.arange(n_students), n_semesters)
semester = np.tile(np.arange(1, n_semesters + 1), n_students)

# Study group: assigned at student level (doesn't change across semesters)
study_group = np.random.choice([0, 1], size=n_students)
study_group_repeated = study_group[student_id]

# Course load varies by semester (12-18 credits)
course_load = np.random.uniform(12, 18, size=N)

# --- The TRUE data-generating process ---
beta_intercept = 3.0
beta_study_group = 0.15   # true treatment effect
beta_course_load = -0.04  # heavier load -> slightly lower GPA
sigma_true = 0.3          # residual standard deviation

# TRUE correlation: unstructured, non-monotonic (see Section 5 above)
R_true = np.array([
    [1.00, 0.70, 0.25, 0.20],
    [0.70, 1.00, 0.30, 0.35],
    [0.25, 0.30, 1.00, 0.65],
    [0.20, 0.35, 0.65, 1.00],
])
Sigma_true = sigma_true**2 * R_true

print("True correlation matrix (the pattern we want to recover):")
labels = ['S1', 'S2', 'S3', 'S4']
print(f"{'':4s} " + "  ".join(f"{l:>5s}" for l in labels))
for i, l in enumerate(labels):
    print(f"{l:4s} " + "  ".join(f"{R_true[i,j]:5.2f}" for j in range(4)))
print(f"\nNote: adjacent-semester correlations are 0.70, 0.30, 0.65")
print(f"      AR(1) would force these to be equal. They're not.")

# Generate correlated errors for each student
errors = np.zeros(N)
for s in range(n_students):
    idx = slice(s * n_semesters, (s + 1) * n_semesters)
    errors[idx] = np.random.multivariate_normal(np.zeros(n_semesters), Sigma_true)

# Generate GPA
gpa = (beta_intercept
       + beta_study_group * study_group_repeated
       + beta_course_load * (course_load - 15)  # centered at 15 credits
       + errors)

# Clip to valid GPA range
gpa = np.clip(gpa, 0, 4.0)

# Build DataFrame
df = pd.DataFrame({
    'gpa': gpa,
    'study_group': study_group_repeated,
    'course_load': course_load - 15,  # centered
    'student': student_id,
    'semester': semester,
})

print(f"\nDataset: {N} observations from {n_students} students over {n_semesters} semesters")
print(f"\nFirst student's data:")
print(df[df['student'] == 0][['student', 'semester', 'gpa', 'study_group', 'course_load']].to_string(index=False))
print(f"\nSecond student's data:")
print(df[df['student'] == 1][['student', 'semester', 'gpa', 'study_group', 'course_load']].to_string(index=False))

True correlation matrix (the pattern we want to recover):
        S1     S2     S3     S4
S1    1.00   0.70   0.25   0.20
S2    0.70   1.00   0.30   0.35
S3    0.25   0.30   1.00   0.65
S4    0.20   0.35   0.65   1.00

Note: adjacent-semester correlations are 0.70, 0.30, 0.65
      AR(1) would force these to be equal. They're not.

Dataset: 240 observations from 60 students over 4 semesters

First student's data:
 student  semester      gpa  study_group  course_load
       0         1 2.634032            0     0.645269
       0         2 3.365471            0    -1.976855
       0         3 3.147840            0    -2.609690
       0         4 2.401775            0     2.693313

Second student's data:
 student  semester      gpa  study_group  course_load
       1         1 3.113614            1     2.793792
       1         2 3.093689            1     1.850384
       1         3 3.033765            1    -1.172317
       1         4 2.793780            1    -2.413967


Notice how each student's GPA values cluster together — they don't jump randomly from 2.0 to 4.0 between semesters. That's the correlation at work.

Now let's fit OLS and see what happens:

In [2]:
from python_gls import GLS

# Fit OLS (ignoring the correlation between semesters for the same student)
result_ols = GLS.from_formula(
    "gpa ~ study_group + course_load",
    data=df
).fit()

print("OLS Results (IGNORING within-student correlation)")
print("=" * 55)
print(result_ols.summary())
print(f"\nTrue study group effect: {beta_study_group}")
print(f"OLS estimate:            {result_ols.params['study_group']:.4f}")
print(f"OLS std error:           {result_ols.bse['study_group']:.4f}")
print(f"OLS p-value:             {result_ols.pvalues['study_group']:.4f}")

OLS Results (IGNORING within-student correlation)
                      Generalized Least Squares Results                       
Method:          REML                 Log-Likelihood:         -65.0685
No. Observations:240                  AIC:                    138.1371
Df Model:        2                    BIC:                    152.0596
Df Residuals:    237                  Sigma^2:                0.094690
Converged:       Yes                  Iterations:                    0
------------------------------------------------------------------------------
                           coef    std err          t      P>|t|     [0.025     0.975]
------------------------------------------------------------------------------
           Intercept     3.0048     0.0286   105.1047     0.0000     2.9484     3.0611
         study_group     0.1056     0.0398     2.6545     0.0085     0.0272     0.1840
         course_load    -0.0381     0.0112    -3.4109     0.0008    -0.0602    -0.0161

True stud

OLS reports standard errors as if all 240 observations were independent. But they're not — 4 observations come from each of 60 students, and observations within a student are correlated.

A common first attempt: fit GLS with AR(1), which assumes correlation decays smoothly with time distance. It's better than OLS, but it's the wrong structure for our data.

In [3]:
from python_gls import GLS
from python_gls.correlation import CorAR1

# Fit GLS with AR(1) — a common default for longitudinal data
result_gls_ar1 = GLS.from_formula(
    "gpa ~ study_group + course_load",
    data=df,
    correlation=CorAR1(),   # Assumes correlation decays with time lag
    groups="student",       # Each student is an independent cluster
).fit()

print("GLS Results with AR(1) correlation")
print("=" * 55)
print(result_gls_ar1.summary())
print(f"\nAR(1) estimated phi: {result_gls_ar1.correlation_params[0]:.3f}")
print(f"  Implied correlations: lag 1 = {result_gls_ar1.correlation_params[0]:.3f}, "
      f"lag 2 = {result_gls_ar1.correlation_params[0]**2:.3f}, "
      f"lag 3 = {result_gls_ar1.correlation_params[0]**3:.3f}")
print(f"\nBut the TRUE adjacent correlations are 0.70, 0.30, 0.65 — not equal!")
print(f"AR(1) can only fit a single decay rate. It compromises.")

GLS Results with AR(1) correlation
                      Generalized Least Squares Results                       
Method:          REML                 Log-Likelihood:         -27.5374
No. Observations:240                  AIC:                     65.0748
Df Model:        2                    BIC:                     82.4780
Df Residuals:    237                  Sigma^2:                0.096010
Converged:       Yes                  Iterations:                    5
------------------------------------------------------------------------------
                           coef    std err          t      P>|t|     [0.025     0.975]
------------------------------------------------------------------------------
           Intercept     2.9955     0.0432    69.2737     0.0000     2.9103     3.0807
         study_group     0.1136     0.0602     1.8885     0.0602    -0.0049     0.2321
         course_load    -0.0385     0.0086    -4.4897     0.0000    -0.0554    -0.0216
Correlation Structure: Co

### OLS vs GLS (AR1): A First Comparison

AR(1) improves on OLS — the standard errors are more honest and AIC is lower. But it's fitting the wrong correlation structure to our data. We'll see in Section 7 that the unstructured model does better.

In [4]:
print(f"{'':30s} {'OLS':>12s} {'GLS (AR1)':>12s} {'Truth':>12s}")
print("-" * 68)
print(f"{'study_group coefficient':30s} {result_ols.params['study_group']:12.4f} {result_gls_ar1.params['study_group']:12.4f} {beta_study_group:12.4f}")
print(f"{'study_group std error':30s} {result_ols.bse['study_group']:12.4f} {result_gls_ar1.bse['study_group']:12.4f} {'':>12s}")
print(f"{'study_group p-value':30s} {result_ols.pvalues['study_group']:12.4f} {result_gls_ar1.pvalues['study_group']:12.4f} {'':>12s}")
print(f"{'course_load coefficient':30s} {result_ols.params['course_load']:12.4f} {result_gls_ar1.params['course_load']:12.4f} {beta_course_load:12.4f}")
print(f"{'AIC':30s} {result_ols.aic:12.1f} {result_gls_ar1.aic:12.1f} {'':>12s}")
print(f"\nGLS standard errors are larger (more honest) because they account")
print(f"for the fact that correlated observations carry less information.")
print(f"\nBut AR(1) is the WRONG structure here. Can we do better?")

                                        OLS    GLS (AR1)        Truth
--------------------------------------------------------------------
study_group coefficient              0.1056       0.1136       0.1500
study_group std error                0.0398       0.0602             
study_group p-value                  0.0085       0.0602             
course_load coefficient             -0.0381      -0.0385      -0.0400
AIC                                   138.1         65.1             

GLS standard errors are larger (more honest) because they account
for the fact that correlated observations carry less information.

But AR(1) is the WRONG structure here. Can we do better?


**What we've seen so far:**

1. **GLS with AR(1) is better than OLS** — larger, more honest standard errors and lower AIC.
2. **But AR(1) is the wrong structure.** It forces adjacent correlations to be equal (all $\phi$), while our data has correlations of 0.70, 0.30, and 0.65 between adjacent semesters.
3. **We need a model that can recover the true non-monotonic pattern.** That's the unstructured model (`CorSymm`).

## 6. Correlation Structures: How Data Points Talk to Each Other

The key insight of GLS is that the correlation between observations is not a nuisance to be ignored — it's a **structure to be modeled**. Different structures encode different beliefs about *how* observations within a group relate to each other.

### Compound Symmetry (Exchangeable)

The simplest structure: any two observations from the same student are equally correlated, regardless of how far apart in time they are.

$$\text{Cor}(Y_{i,t}, Y_{i,s}) = \rho \quad \text{for all } t \neq s$$

The correlation matrix for 4 semesters looks like:

$$R = \begin{pmatrix} 1 & \rho & \rho & \rho \\ \rho & 1 & \rho & \rho \\ \rho & \rho & 1 & \rho \\ \rho & \rho & \rho & 1 \end{pmatrix}$$

This says: your GPA in semester 1 is just as correlated with semester 4 as with semester 2. That might be reasonable for some data (e.g., test-retest reliability), but for longitudinal data it's often too restrictive — nearby time points are usually more correlated than distant ones (Snijders & Bosker, 2012, Sec. 15.1.1).

### AR(1): Autoregressive

A more realistic structure for time-ordered data: correlation decays with distance.

$$\text{Cor}(Y_{i,t}, Y_{i,s}) = \phi^{|t - s|}$$

With $\phi = 0.6$ and 4 semesters:

$$R = \begin{pmatrix} 1 & 0.6 & 0.36 & 0.216 \\ 0.6 & 1 & 0.6 & 0.36 \\ 0.36 & 0.6 & 1 & 0.6 \\ 0.216 & 0.36 & 0.6 & 1 \end{pmatrix}$$

Adjacent semesters have correlation 0.6, but semesters 1 and 4 have correlation only $0.6^3 = 0.216$. This captures the intuition that recent semesters are more predictive of your current GPA than distant ones.

As Snijders and Bosker (2012, Sec. 15.3) describe, autocorrelated residuals arise naturally in longitudinal data. When the random intercept and slope model doesn't fully capture the within-individual dynamics, the level-one residuals often show a pattern where a positive deviation in one period tends to be followed by another positive deviation — exactly the AR(1) pattern.

### Unstructured: Let the Data Decide

What if the true correlation doesn't follow any simple pattern? Maybe semesters 2 and 3 are highly correlated (both fall/spring of the same academic year) but semesters 1 and 3 are weakly correlated (different academic years). A rigid structure like AR(1) can't capture this.

The **unstructured** (or **general**) correlation matrix estimates every pairwise correlation freely:

$$R = \begin{pmatrix} 1 & r_{12} & r_{13} & r_{14} \\ r_{12} & 1 & r_{23} & r_{24} \\ r_{13} & r_{23} & 1 & r_{34} \\ r_{14} & r_{24} & r_{34} & 1 \end{pmatrix}$$

Each $r_{ts}$ is estimated separately from the data. This is the most flexible approach — no pattern is assumed. The trade-off is that it requires estimating $d(d-1)/2$ parameters (6 for 4 time points), which means you need enough data.

This is what Snijders and Bosker (2012, Sec. 15.1.3) call the **fully multivariate model**, where the covariance matrix is left "completely free." It serves as a benchmark: if a simpler structure (like AR(1) or compound symmetry) fits almost as well, prefer the simpler model. If not, the unstructured model reveals patterns that simpler models miss.

## 7. The Right Structure: Letting the Data Speak

Our data has a non-monotonic correlation pattern that neither compound symmetry nor AR(1) can capture. Only the unstructured model (`CorSymm`) can fit all six pairwise correlations freely. Let's compare all three.

In [5]:
from python_gls.correlation import CorAR1, CorCompSymm, CorSymm

# Fit three models with different correlation structures

# 1. Compound Symmetry: one parameter (rho), all pairs equal
result_cs = GLS.from_formula(
    "gpa ~ study_group + course_load",
    data=df,
    correlation=CorCompSymm(),
    groups="student",
).fit()

# 2. AR(1): one parameter (phi), correlation decays with lag
result_ar1 = GLS.from_formula(
    "gpa ~ study_group + course_load",
    data=df,
    correlation=CorAR1(),
    groups="student",
).fit()

# 3. Unstructured (CorSymm): d(d-1)/2 = 6 free parameters
result_sym = GLS.from_formula(
    "gpa ~ study_group + course_load",
    data=df,
    correlation=CorSymm(),
    groups="student",
).fit()

print("Model Comparison")
print("=" * 75)
print(f"{'Model':25s} {'AIC':>10s} {'BIC':>10s} {'Log-Lik':>10s} {'Corr Params':>15s}")
print("-" * 75)
print(f"{'OLS (none)':25s} {result_ols.aic:10.1f} {result_ols.bic:10.1f} {result_ols.loglik:10.1f} {'0':>15s}")
print(f"{'Compound Symmetry':25s} {result_cs.aic:10.1f} {result_cs.bic:10.1f} {result_cs.loglik:10.1f} {'1 (rho)':>15s}")
print(f"{'AR(1)':25s} {result_ar1.aic:10.1f} {result_ar1.bic:10.1f} {result_ar1.loglik:10.1f} {'1 (phi)':>15s}")
print(f"{'Unstructured (CorSymm)':25s} {result_sym.aic:10.1f} {result_sym.bic:10.1f} {result_sym.loglik:10.1f} {'6 (r_ij)':>15s}")
print(f"\nLowest AIC wins. The unstructured model should win here because the")
print(f"true correlation is non-monotonic and can't be captured by AR(1) or CS.")

Model Comparison
Model                            AIC        BIC    Log-Lik     Corr Params
---------------------------------------------------------------------------
OLS (none)                     138.1      152.1      -65.1               0
Compound Symmetry               69.3       86.7      -29.7         1 (rho)
AR(1)                           65.1       82.5      -27.5         1 (phi)
Unstructured (CorSymm)          52.1       86.9      -16.1        6 (r_ij)

Lowest AIC wins. The unstructured model should win here because the
true correlation is non-monotonic and can't be captured by AR(1) or CS.


In [6]:
# What parameters does each structure estimate?

print("=" * 70)
print("PARAMETER NAMES AND ESTIMATES BY STRUCTURE")
print("=" * 70)

# --- Compound Symmetry ---
rho = result_cs.correlation_params[0]
print(f"\n1. Compound Symmetry (CorCompSymm)")
print(f"   Parameter: rho = {rho:.3f}")
print(f"   Meaning: ALL pairs have correlation {rho:.3f}")
print(f"   Implied correlation matrix:")
print(f"        S1    S2    S3    S4")
for i, l in enumerate(['S1','S2','S3','S4']):
    row = [1.0 if i==j else rho for j in range(4)]
    print(f"   {l}  " + "  ".join(f"{v:.2f}" for v in row))

# --- AR(1) ---
phi = result_ar1.correlation_params[0]
print(f"\n2. AR(1) (CorAR1)")
print(f"   Parameter: phi = {phi:.3f}")
print(f"   Meaning: corr(t, s) = phi^|t-s|, correlation decays with distance")
print(f"   Implied correlation matrix:")
print(f"        S1    S2    S3    S4")
for i, l in enumerate(['S1','S2','S3','S4']):
    row = [phi**abs(i-j) for j in range(4)]
    print(f"   {l}  " + "  ".join(f"{v:.2f}" for v in row))

# --- Unstructured ---
print(f"\n3. Unstructured (CorSymm)")
print(f"   Parameters: {n_semesters*(n_semesters-1)//2} free pairwise correlations")
R_est = result_sym.model.correlation.get_correlation_matrix(n_semesters)
print(f"   Estimated correlation matrix:")
print(f"        S1    S2    S3    S4")
for i, l in enumerate(['S1','S2','S3','S4']):
    print(f"   {l}  " + "  ".join(f"{R_est[i,j]:.2f}" for j in range(4)))

# --- Compare to truth ---
print(f"\n{'=' * 70}")
print(f"ESTIMATED vs TRUE CORRELATIONS")
print(f"{'=' * 70}")
print(f"{'Pair':>10s} {'True':>8s} {'CS':>8s} {'AR(1)':>8s} {'CorSymm':>8s}")
print(f"{'-'*10} {'-'*8} {'-'*8} {'-'*8} {'-'*8}")
pairs = [(0,1,'S1-S2'), (0,2,'S1-S3'), (0,3,'S1-S4'),
         (1,2,'S2-S3'), (1,3,'S2-S4'), (2,3,'S3-S4')]
for i, j, label in pairs:
    true_r = R_true[i,j]
    cs_r = rho
    ar_r = phi**abs(i-j)
    sym_r = R_est[i,j]
    print(f"{label:>10s} {true_r:8.3f} {cs_r:8.3f} {ar_r:8.3f} {sym_r:8.3f}")
print(f"\nOnly CorSymm can recover the non-monotonic pattern:")
print(f"  S1-S2 = 0.70 (high), S2-S3 = 0.30 (low), S3-S4 = 0.65 (high)")

PARAMETER NAMES AND ESTIMATES BY STRUCTURE

1. Compound Symmetry (CorCompSymm)
   Parameter: rho = 0.508
   Meaning: ALL pairs have correlation 0.508
   Implied correlation matrix:
        S1    S2    S3    S4
   S1  1.00  0.51  0.51  0.51
   S2  0.51  1.00  0.51  0.51
   S3  0.51  0.51  1.00  0.51
   S4  0.51  0.51  0.51  1.00

2. AR(1) (CorAR1)
   Parameter: phi = 0.591
   Meaning: corr(t, s) = phi^|t-s|, correlation decays with distance
   Implied correlation matrix:
        S1    S2    S3    S4
   S1  1.00  0.59  0.35  0.21
   S2  0.59  1.00  0.59  0.35
   S3  0.35  0.59  1.00  0.59
   S4  0.21  0.35  0.59  1.00

3. Unstructured (CorSymm)
   Parameters: 6 free pairwise correlations
   Estimated correlation matrix:
        S1    S2    S3    S4
   S1  1.00  0.69  0.40  0.35
   S2  0.69  1.00  0.40  0.54
   S3  0.40  0.40  1.00  0.67
   S4  0.35  0.54  0.67  1.00

ESTIMATED vs TRUE CORRELATIONS
      Pair     True       CS    AR(1)  CorSymm
---------- -------- -------- -------- ------

The table above makes the point clearly:

- **Compound Symmetry** (`rho`) forces all six correlations to be identical. It can't distinguish S1-S2 (0.70) from S2-S3 (0.30).
- **AR(1)** (`phi`) forces correlations to decay monotonically with lag. It gets the overall level roughly right, but it can't capture the dip at S2-S3 followed by the rebound at S3-S4.
- **Unstructured** (`CorSymm`) estimates each of the 6 pairwise correlations freely. It's the only structure that recovers the true non-monotonic pattern.

The cost of this flexibility: 6 parameters instead of 1. AIC penalizes for this extra complexity, but if the true structure is genuinely irregular, CorSymm still wins.

## 8. The Math Behind GLS (Accessible Version)

OLS finds coefficients by minimizing the sum of squared errors:

$$\hat{\beta}_{\text{OLS}} = (X'X)^{-1}X'y$$

GLS does the same, but it **weights** the observations by the inverse of their covariance structure:

$$\hat{\beta}_{\text{GLS}} = (X'\Omega^{-1}X)^{-1}X'\Omega^{-1}y$$

where $\Omega$ is the covariance matrix that encodes the correlations. Think of it this way:
- Observations that are highly correlated with others get **less weight** (they provide redundant information)
- Observations that are relatively independent get **more weight** (they provide unique information)

The covariance matrix has a specific structure (Snijders & Bosker, 2012, Ch. 15):

$$\Omega_g = \sigma^2 \cdot A_g^{1/2} \, R_g \, A_g^{1/2}$$

where:
- $R_g$ is the **correlation matrix** (AR(1), compound symmetry, unstructured, etc.)
- $A_g$ is a diagonal matrix of **variance weights** (for heteroscedasticity)
- $\sigma^2$ is the overall residual variance

The key innovation in GLS is that $R_g$ and $A_g$ are **estimated from the data** via maximum likelihood (ML) or restricted maximum likelihood (REML). You don't need to know the correlation in advance — the algorithm discovers it.

### ML vs REML

Two estimation methods are available:
- **ML** (Maximum Likelihood): tends to underestimate variance (divides by $N$)
- **REML** (Restricted Maximum Likelihood): corrects for the bias (divides by $N - k$), where $k$ is the number of fixed effects

REML is the default in both R's `nlme::gls()` and in `python-gls`, and is generally preferred for estimating variance parameters (Snijders & Bosker, 2012, Sec. 4.7).

## 9. Beyond Correlation: Heteroscedasticity

Sometimes the *variance* of the errors isn't constant either. For example:
- GPA variability might be larger for students with heavier course loads (more ways to do well or poorly)
- Measurement precision might differ between online and in-person semesters

GLS can handle this too, by modeling the variance as a function of a covariate or group membership. `python-gls` provides six variance functions:

In [7]:
from python_gls.variance import VarIdent

# What if the variance differs between study group and non-study-group students?
result_hetero = GLS.from_formula(
    "gpa ~ study_group + course_load",
    data=df,
    correlation=CorAR1(),
    variance=VarIdent("study_group"),  # Allow different variance per group
    groups="student",
).fit()

print("GLS with AR(1) + Group-specific Variance")
print("=" * 50)
print(result_hetero.summary())
print(f"\nVariance ratio parameter: {result_hetero.variance_params}")
print(f"(A value near 1.0 means variances are similar across groups)")

GLS with AR(1) + Group-specific Variance
                      Generalized Least Squares Results                       
Method:          REML                 Log-Likelihood:         -27.2646
No. Observations:240                  AIC:                     66.5292
Df Model:        2                    BIC:                     87.4130
Df Residuals:    237                  Sigma^2:                0.103701
Converged:       Yes                  Iterations:                    6
------------------------------------------------------------------------------
                           coef    std err          t      P>|t|     [0.025     0.975]
------------------------------------------------------------------------------
           Intercept     2.9953     0.0451    66.3899     0.0000     2.9064     3.0842
         study_group     0.1139     0.0608     1.8732     0.0623    -0.0059     0.2336
         course_load    -0.0378     0.0085    -4.4333     0.0000    -0.0546    -0.0210
Correlation Structu

## 10. Introducing `python-gls`: GLS for Python, Finally

### The Gap

For over 25 years, R users have had `nlme::gls()` (Pinheiro & Bates, 2000) — a function that estimates correlation and variance structures from data automatically. It has been the workhorse for longitudinal data analysis in biostatistics, social science, and beyond.

Python, despite its dominance in data science, has had **no equivalent**. The `statsmodels.GLS` class requires you to supply a pre-computed covariance matrix — you have to know $\Omega$ in advance, which defeats the purpose. There has been no way to say "I have AR(1) correlated errors; please estimate $\phi$ for me."

**`python-gls` fills this gap.** It is the first Python library to provide automatic estimation of correlation and variance structures via ML/REML, matching R's `nlme::gls()` behavior.

### Features at a Glance

| Feature | Details |
|---|---|
| **11 correlation structures** | AR(1), ARMA(p,q), continuous-time AR(1), compound symmetry, unstructured, exponential, Gaussian, linear, rational quadratic, spherical |
| **6 variance functions** | Group-specific, power, exponential, constant-power, fixed, combined |
| **Estimation** | ML and REML, with profile likelihood optimization |
| **R-style formulas** | `"y ~ x1 + C(group) * time"` — same syntax you'd use in R |
| **Parallel computation** | `n_jobs=-1` distributes group-level operations across CPU cores |
| **Validated against R** | Comprehensive test suite comparing output to R's `nlme::gls()` |

### Performance: Parallel GLS

Real datasets can be large — thousands of groups, each with dozens of time points. `python-gls` includes several optimizations to make GLS fast:

1. **Analytic inverses** for AR(1) and compound symmetry matrices — $O(m)$ instead of $O(m^3)$
2. **Batched NumPy operations** for balanced panels — a single matrix multiply across all groups
3. **Block-diagonal inversion** — $O(n \cdot m^3)$ instead of $O(N^3)$
4. **Thread-level parallelism** — pass `n_jobs=-1` to distribute work across CPU cores

In [8]:
import time

# Generate a larger dataset to see parallelism benefits
np.random.seed(123)
n_large = 500
n_t = 8
N_large = n_large * n_t

df_large = pd.DataFrame({
    'y': np.random.randn(N_large),
    'x': np.random.randn(N_large),
    'group': np.repeat(np.arange(n_large), n_t),
})
# Add a real signal
df_large['y'] = 1.0 + 0.5 * df_large['x'] + df_large['y'] * 0.5

# Sequential
t0 = time.time()
r_seq = GLS.from_formula("y ~ x", data=df_large, correlation=CorAR1(), groups="group").fit(n_jobs=1)
t_seq = time.time() - t0

# Parallel
t0 = time.time()
r_par = GLS.from_formula("y ~ x", data=df_large, correlation=CorAR1(), groups="group").fit(n_jobs=-1)
t_par = time.time() - t0

print(f"Dataset: {N_large} observations, {n_large} groups, {n_t} time points")
print(f"Sequential (n_jobs=1):  {t_seq:.2f}s")
print(f"Parallel   (n_jobs=-1): {t_par:.2f}s")
print(f"Speedup: {t_seq/t_par:.1f}x")

Dataset: 4000 observations, 500 groups, 8 time points
Sequential (n_jobs=1):  0.07s
Parallel   (n_jobs=-1): 0.07s
Speedup: 1.0x


## 11. Putting It All Together: A Complete Workflow

Here's the recommended workflow for analyzing longitudinal or clustered data with `python-gls`:

In [9]:
# Step 1: Start with OLS as a baseline
result_ols = GLS.from_formula("gpa ~ study_group + course_load", data=df).fit()

# Step 2: Try different correlation structures
result_cs  = GLS.from_formula("gpa ~ study_group + course_load", data=df,
                               correlation=CorCompSymm(), groups="student").fit()
result_ar1 = GLS.from_formula("gpa ~ study_group + course_load", data=df,
                               correlation=CorAR1(), groups="student").fit()
result_sym = GLS.from_formula("gpa ~ study_group + course_load", data=df,
                               correlation=CorSymm(), groups="student").fit()

# Step 3: Compare models using AIC (lower is better)
models = [
    ("OLS",                result_ols),
    ("Compound Symmetry",  result_cs),
    ("AR(1)",              result_ar1),
    ("Unstructured",       result_sym),
]

print("Step 3: Model Selection via AIC")
print("=" * 50)
for name, r in sorted(models, key=lambda x: x[1].aic):
    print(f"  {name:25s}  AIC = {r.aic:8.1f}  {'<-- best' if r.aic == min(m[1].aic for m in models) else ''}")

# Step 4: Interpret the best model
best_name, best_result = min(models, key=lambda x: x[1].aic)
print(f"\nStep 4: Results from best model ({best_name})")
print("=" * 50)
print(best_result.summary())
print(f"\n95% Confidence Intervals:")
print(best_result.conf_int())

Step 3: Model Selection via AIC
  Unstructured               AIC =     52.1  <-- best
  AR(1)                      AIC =     65.1  
  Compound Symmetry          AIC =     69.3  
  OLS                        AIC =    138.1  

Step 4: Results from best model (Unstructured)
                      Generalized Least Squares Results                       
Method:          REML                 Log-Likelihood:         -16.0718
No. Observations:240                  AIC:                     52.1435
Df Model:        2                    BIC:                     86.9499
Df Residuals:    237                  Sigma^2:                0.096054
Converged:       Yes                  Iterations:                    9
------------------------------------------------------------------------------
                           coef    std err          t      P>|t|     [0.025     0.975]
------------------------------------------------------------------------------
           Intercept     3.0074     0.0456    65.

## 12. Summary

### What You've Learned

1. **OLS assumes independence** — but real data (especially longitudinal or clustered data) violates this. Ignoring the violation leads to standard errors that are too small and conclusions that are unreliable.

2. **Correlation structures** describe how observations within a group are related. Common choices include:
   - **Compound symmetry**: all pairs equally correlated (simplest, most restrictive)
   - **AR(1)**: correlation decays with time lag (natural for time series)
   - **Unstructured**: every pair has its own correlation (most flexible, lets data speak)

3. **Unstructured correlations** are especially valuable when the pattern of association between time points doesn't follow a simple formula. While mixed effects models (random intercepts/slopes) impose specific structures, GLS with `CorSymm` can capture arbitrary patterns.

4. **GLS estimates these structures from your data** via maximum likelihood — you don't need to know the correlation in advance.

5. **Model selection** (AIC/BIC) helps you choose the right level of complexity: start simple, and add complexity only when the data supports it.

### `python-gls` in One Paragraph

`python-gls` is the first Python library to provide automatic GLS estimation with learned correlation and variance structures — functionality that has been available in R through `nlme::gls()` (Pinheiro & Bates, 2000) for over 25 years but has been missing from the Python ecosystem. It supports 11 correlation structures (including unstructured), 6 variance functions, both ML and REML estimation, R-style formula syntax, and parallel computation for large datasets. It has been validated against R's `nlme::gls()` across 12 diverse test scenarios.

Install it with:
```bash
pip install python-gls
```

## References

- Dorman, C. F. (2008). Effects of incorporating spatial autocorrelation into the analysis of species distribution data. *Global Ecology and Biogeography*, 16(2), 129–138.

- Hedges, L. V., & Hedberg, E. C. (2007). Intraclass correlation values for planning group-randomized trials in education. *Educational Evaluation and Policy Analysis*, 29(1), 60–87.

- Pinheiro, J. C., & Bates, D. M. (2000). *Mixed-Effects Models in S and S-PLUS*. Springer.

- Pinheiro, J. C., & Bates, D. M. (1996). Unconstrained parametrizations for variance-covariance matrices. *Statistics and Computing*, 6(3), 289–296.

- Snijders, T. A. B., & Bosker, R. J. (2012). *Multilevel Analysis: An Introduction to Basic and Advanced Multilevel Modeling* (2nd ed.). SAGE Publications.
  - Ch. 2: Dependence as a nuisance vs. an interesting phenomenon
  - Ch. 3, Sec. 3.3–3.4: Intraclass correlation and design effects
  - Ch. 15, Sec. 15.1: Fixed occasion designs, compound symmetry
  - Ch. 15, Sec. 15.1.3: The fully multivariate (unstructured) model
  - Ch. 15, Sec. 15.3: Autocorrelated residuals